In [1]:
import sys
sys.path.append('..')

import os
import psutil

import numpy as np
np.random.seed(0)

In [2]:
import matplotlib.pyplot as plt
from pylab import *
from keras.models import Sequential
from keras import optimizers
from keras.models import load_model
from keras.optimizers import Adam, SGD
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Dropout, Convolution2D
from keras.models import Model
from keras.datasets import cifar10

import random
import itertools

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense

from keras.preprocessing.image import ImageDataGenerator
from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K
from keras.utils import plot_model,normalize

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from custom_lib.triplet_utils import buildDataSet, build_model
from custom_lib.triplet_utils import get_batch_hard, compute_probs
from custom_lib.triplet_utils import add_top, remove_top

from art.classifiers import KerasClassifier
from art.attacks.projected_gradient_descent import ProjectedGradientDescent
from art.attacks.iterative_method import BasicIterativeMethod
from art.defences.adversarial_trainer import AdversarialTrainer
from art.attacks.fast_gradient import FastGradientMethod

from custom_lib.build_resnet import resnet_v1, resnet_emb

Using TensorFlow backend.


In [4]:
def restrict_cpu():
    p = psutil.Process()

    for i in p.threads():
        temp = psutil.Process(i.id)

        temp.cpu_affinity([i for i in range(4)])

In [9]:
from keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train / 255.
x_test = x_test / 255.

val_train = to_categorical(y_train, 10)
val_test = to_categorical(y_test, 10)

In [5]:
import sys
sys.path.append('..')
import itertools


from custom_lib.triplet_utils import buildDataSet

nb_classes = 10
img_rows, img_cols = 32, 32
input_shape = (img_rows, img_cols, 3)
in_shape = input_shape

dataset_train, dataset_test, x_train_origin, y_train_origin, x_test_origin, y_test_origin = buildDataSet(cifar=True)

dataset_train = np.array(list(itertools.chain.from_iterable(dataset_train)))
dataset_test = np.array(list(itertools.chain.from_iterable(dataset_test)))

In [133]:
s1 = random.sample(range(50_000), 64)
s2 = []
sy = []

for i in s1:
    label = i // 5000
    n = random.randint(5000 * label, (5000 * (label + 1)) - 1)
    if n == i:
        n += 1
    s2.append(n)
    
c1 = dataset_train[s1]
c2 = np.copy(c1)
c3 = dataset_train[s2]

batch = np.array(list(zip(c1, c2, c3)))

In [9]:
import random
import numpy as np
import matplotlib.pyplot as plt

In [74]:
class_idx = random.randint(0, 9)
idxs = list(range(5000 * class_idx, (5000 * (class_idx + 1))))
np.random.shuffle(idxs)
imgs = dataset_train[idxs]

In [80]:
dataset_train.shape

(50000, 32, 32, 3)

In [84]:
from keras.utils import to_categorical

In [85]:
dataset_y = np.array([i // 5000 for i in range(50000)])

dataset_y_c = to_categorical(dataset_y, 10)

In [90]:
dataset_y_c[15100]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)